In [2]:
from google.colab import files
uploaded = files.upload()


Saving holdings.csv to holdings.csv
Saving trades.csv to trades.csv


In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
print("Key loaded:", bool(OPENAI_API_KEY))


Key loaded: True


In [4]:
!pip install openai==0.28

import pandas as pd
import numpy as np
import os
from datetime import datetime
import openai

openai.api_key = OPENAI_API_KEY


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 2.14.0
    Uninstalling openai-2.14.0:
      Successfully uninstalled openai-2.14.0


In [62]:
holdings_df = pd.read_csv("holdings.csv")
trades_df   = pd.read_csv("trades.csv")

# Basic sanity check
print(holdings_df.shape, trades_df.shape)

# Parse dates + add Year
holdings_df["AsOfDate"] = pd.to_datetime(holdings_df["AsOfDate"], errors="coerce")
trades_df["TradeDate"]  = pd.to_datetime(trades_df["TradeDate"], errors="coerce")
holdings_df["Year"] = holdings_df["AsOfDate"].dt.year
trades_df["Year"]   = trades_df["TradeDate"].dt.year

# Schema for LLM
holdings_cols = holdings_df.columns.tolist()
trades_cols   = trades_df.columns.tolist()


(1022, 25) (649, 31)


In [56]:
def llm_to_query(question):
    prompt = f"""
You are a data engineer.

We have two pandas DataFrames:

holdings_df with columns:
{holdings_cols}

trades_df with columns:
{trades_cols}

Rules:
- Use ONLY these columns exactly as written
- Use only holdings_df or trades_df
- Assign final answer to variable: result
- If question cannot be answered from these tables, return exactly: NONE
- Return only python pandas code, no explanation

Question: {question}
"""

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user","content":prompt}],
        temperature=0
    )

    return response.choices[0].message["content"].strip()


In [7]:
def run_query_from_llm(code):
    if code == "NONE":
        return None

    local_scope = {
        "holdings_df": holdings_df,
        "trades_df": trades_df
    }

    try:
        exec(code, {}, local_scope)
        return local_scope.get("result", None)
    except Exception as e:
        print("Execution error:", e)
        return None


In [8]:
def llm_explain(question, data):
    prompt = f"""
You are a data assistant.
Answer ONLY using this data.
If data is not enough, say: Sorry can not find the answer.

Question: {question}
Data:
{data}
"""

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role":"user","content":prompt}],
        temperature=0
    )

    return response.choices[0].message["content"]


In [76]:
def chatbot(question):
    print("Question: ", question)
    code = llm_to_query(question)
    # print("Generated Code:\n", code)

    data = run_query_from_llm(code)

    if data is None:
        return "Sorry can not find the answer"

    if hasattr(data, "empty") and data.empty:
        return "Sorry can not find the answer"

    if hasattr(data, "size") and data.size == 0:
        return "Sorry can not find the answer"

    if isinstance(data, (int, float)) and pd.isna(data):
        return "Sorry can not find the answer"

    if isinstance(data, str) and data.strip().upper() == "NONE":
        return "Sorry can not find the answer"

    # 🔥 If result is simple (string, number, Series), return directly
    if isinstance(data, (str, int, float)) or hasattr(data, "shape"):
        return data

    # Only use LLM for complex tables
    return llm_explain(question, data)


In [18]:
holdings_df["PortfolioName"].unique()[:10]



array(['Garfield', 'Heather', 'MNC Investment Fund', 'Northpoint 401K',
       'CoYold 1', 'Opium Holdings Partners', 'Ytum', 'Platpot',
       'Hi Yield', 'Warren Lee IG'], dtype=object)

In [79]:
fund = holdings_df["PortfolioName"].dropna().unique()[0]

print(chatbot("How many security types we have inside trades sheet and name them?"))
print()

print(chatbot("Which fund has the highest total market value in base currency?"))
print()

print(chatbot("Which fund holds the highest number of unique securities and how many in number"))
print()

print(chatbot("How many rows of data we have for 'MNC Investment Fund'?"))
print()

print(chatbot("What are the top 5 securities by market value across all funds?"))
print()

print(chatbot("Which fund has the highest average price per security?"))
print()

print(chatbot("Which fund gave maximum returns in year 2023?"))
print()

print(chatbot("Who is the USA president?"))





Question:  How many security types we have inside trades sheet and name them?
We have 15 security types inside the trades sheet. They are: Equity, Option, Bond, AssetBacked, FX Forward, FX Option, IR Swap, Loan, CDS Contract, Repo Contract, CDO Tranche, Preferred, Future, Swaption, Total Return Swap.

Question:  Which fund has the highest total market value in base currency?
Platpot

Question:  Which fund holds the highest number of unique securities and how many in number
The MNC Investment Fund holds the highest number of unique securities with 157 in number.

Question:  How many rows of data we have for 'MNC Investment Fund'?
243

Question:  What are the top 5 securities by market value across all funds?
SecName
FxFwd EURUSD 8/11/2021 CS    97115004.0
AA                           93796200.0
SAP GR                       83482211.4
F                            83162500.0
912828PX2                    70841200.0
Name: MV_Base, dtype: float64

Question:  Which fund has the highest averag